In [ ]:
!pip install torch torchaudio transformers librosa scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
import os
import numpy as np
import librosa
import torch
from transformers import Wav2Vec2FeatureExtractor, HubertModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Load HuBERT model
model_name = "facebook/hubert-base-ls960"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = HubertModel.from_pretrained(model_name)

# Dataset paths
DATASET_PATH = "/content/drive/MyDrive/Pneumonia_Project/Datasets/Audio/Audio_Final_Dataset"
PNEUMONIA_PATH = os.path.join(DATASET_PATH, "Pneumonia")
NON_PNEUMONIA_PATH = os.path.join(DATASET_PATH, "Non-Pneumonia")

def extract_features(audio_file):
    """Extract HuBERT embeddings from an audio file."""
    try:
        audio, sr = librosa.load(audio_file, sr=16000)
        input_values = feature_extractor(audio, sampling_rate=16000, return_tensors="pt").input_values
        with torch.no_grad():
            outputs = model(input_values).last_hidden_state
        return torch.mean(outputs, dim=1).squeeze().numpy()  # Average pooling
    except Exception as e:
        print(f"Error processing {audio_file}: {e}")
        return None

# Load dataset
def load_dataset():
    data, labels = [], []

    for label, folder in enumerate([NON_PNEUMONIA_PATH, PNEUMONIA_PATH]):
        for file in os.listdir(folder):
            if file.endswith(".wav"):
                file_path = os.path.join(folder, file)
                embedding = extract_features(file_path)
                if embedding is not None:
                    data.append(embedding)
                    labels.append(label)

    return np.array(data), np.array(labels)

# Prepare data
X, y = load_dataset()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier (SVM)
svm = SVC(kernel="linear", probability=True)
svm.fit(X_train, y_train)

# Evaluate model
y_pred = svm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/feature_extraction_wav2vec2.py:98: RuntimeWarning: Mean of empty slice.
  normed_input_values = [(x - x.mean()) / np.sqrt(x.var() + 1e-7) for x in input_values]
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/feature_extraction_wav2vec2.py:98: RuntimeWarning: Degrees of freedom <= 0 for slice
  normed_input_values = [(x - x.mean()) / np.sqrt(x.var() + 1e-7) for x in input_values]
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Error processing /content/drive/MyDrive/Pneumonia_Project/Datasets/Audio/Audio_Final_Dataset/Pneumonia/6T43bddKoKfG7MwnJWvrPZSsyrc2_breathing-shallow.wav: Calculated padded input size per channel: (0). Kernel size: (10). Kernel size can't be greater than actual input size
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       244
           1       0.82      0.64      0.72        14

    accuracy                           0.97       258
   macro avg       0.90      0.82      0.85       258
weighted avg       0.97      0.97      0.97       258

Accuracy: 0.9728682170542635
